In [17]:
from folktables.acs import adult_filter
from folktables import ACSDataSource, BasicProblem, generate_categories
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, matthews_corrcoef
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.model_selection import KFold
import scipy.optimize as opt
import matplotlib.pyplot as plt
import pandas as pd


### 1. Load and Preprocess the data
We are going to work with the [Folktables](https://github.com/socialfoundations/folktables#quick-start-examples) dataset (*you have already worked with it*). I have chosen some variables for you, but you can add more (*if you like to*) - here is the [full list](https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMS_Data_Dictionary_2021.pdf) of variables (some of them do not exist in `ACSDataSource`). 

Today we are going to debias a regression model using the `SEX` variable. Your model should predict the *Total person's income*  (I've digitized  it in  `target_transform=lambda x: x > 25000`, you can choose another threshold).


* If you code is slow - you can subsample data (aka reduce the number of the samples)

In [5]:
data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
acs_data = data_source.get_data(states=["CA"], download=False)

ACSIncomeNew = BasicProblem(
    features=[
        'AGEP', # include AGE
        'COW', # include class of worker
        'SCHL', # include school education
        'WKHP', # include reported working hours
        'SEX', # include sex
        # some random, possibly noisy
        'JWMNP' # travel time to work
    ],
    target='PINCP',
    target_transform=lambda x: x > 25000,    
    group='SEX',
    preprocess=adult_filter,
    postprocess=lambda x: np.nan_to_num(x, -1),
)

Here is a small snippet to get the names of the categorical variables - I convert categoricals into one-hot encoded (*you don't have to, depending on what assumptions you use about the data*). **Don't forget to normalise the continious features (if you plan to use Cross-Validation features should be normalized per fold, aka not in the global table).**

In [ ]:
definition_df = data_source.get_definitions(download=True)
categories = generate_categories(features=ACSIncomeNew.features, definition_df=definition_df)
# Here I convert categoricals into one-hot encoded (you don't have to, depending on what assumptions you use about the data)
features, labels, groups = ACSIncomeNew.df_to_pandas(acs_data, categories=categories, dummies=True)
features = features.fillna(-1) # Fill nulls with -1 which becomes necessary for the optimization
########### Normalize continious features
scaler = ColumnTransformer([("scaler",StandardScaler(), ["AGEP", "WKHP", "JWMNP"])], remainder="passthrough")


features_transformed = pd.DataFrame(
    scaler.fit_transform(features), 
    columns=scaler.get_feature_names_out()
)
features_transformed.head()

,scaler__AGEP,scaler__WKHP,scaler__JWMNP,"remainder__COW_Employee of a private for-profit company or business, or of an individual, for wages, salary, or commissions","remainder__COW_Employee of a private not-for-profit, tax-exempt, or charitable organization",remainder__COW_Federal government employee,"remainder__COW_Local government employee (city, county, etc.)","remainder__COW_Self-employed in own incorporated business, professional practice or farm","remainder__COW_Self-employed in own not incorporated business, professional practice, or farm",remainder__COW_State government employee,...,remainder__SCHL_Grade 9,remainder__SCHL_Kindergarten,remainder__SCHL_Master's degree,remainder__SCHL_No schooling completed,"remainder__SCHL_Nursery school, preschool",remainder__SCHL_Professional degree beyond a bachelor's degree,remainder__SCHL_Regular high school diploma,"remainder__SCHL_Some college, but less than 1 year",remainder__SEX_Female,remainder__SEX_Male
0,-0.855577,0.163863,-1.002953,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,-1.460230,-1.372352,-0.606647,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,1.495849,-2.294081,-0.012189,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,-0.654027,0.163863,-1.002953,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-1.661781,-1.525973,-1.002953,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


Spliting data into train-test. **Again, if you plan to use Cross-Validation then you should normalise features only inside of a fold**.

In [19]:
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    features_transformed.values, labels.values.reshape(-1), groups.values.reshape(-1), test_size=0.3, random_state=0, shuffle=True)

# N = 1000 ### I am subsampling because it is slow on my machine
# X_train = X_train[:N]
# y_train = y_train[:N]

### 2.  Regression model (without Fairness constraints)
Let's first train a simple **Logistic Regression**. 
1. Use L2 penalty to train the model (you should find the optimal value for the regularizer)
2. Calculate the total performance metric
3. Calculate and compare the performance metric for each `SEX` group (use your favourite metric introduced during the course).

In [20]:
X_train

array([[-1.25867905,  0.16386303,  0.78042282, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.55527824,  0.16386303,  0.18596423, ...,  0.        ,
         1.        ,  0.        ],
       [-0.11655779,  0.16386303,  0.18596423, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [ 1.15993067,  1.70007775, -0.52738607, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.28654383,  1.70007775,  1.37488141, ...,  0.        ,
         0.        ,  1.        ],
       [ 1.02556346, -0.60424432, -0.40849436, ...,  0.        ,
         1.        ,  0.        ]])

In [21]:
#scaler = ColumnTransformer([("scaler", StandardScaler(), ["AGEP"])], remainder="passthrough")
# Create the pipeline
logistic_model = Pipeline(steps=[
    ("logi", LogisticRegression(max_iter=5000, penalty="l2", C=0.8497534359086438, tol=1e-4, solver="saga"))
])
# Fit the model
logistic_model.fit(X_train, y_train)

log_y_pred = logistic_model.predict(X_test)
log_y_prob = logistic_model.predict_proba(X_test)[:, 1]

In [24]:
bal_acc = balanced_accuracy_score(y_test, log_y_pred)
bal_acc

0.7621132449937742

### 3. Constrained Regression Model 
Now let's try to include the [Fairness Constraint](https://arxiv.org/abs/1706.02409)! You'll have to implement couple of things from scratch (as it is tricky to add a custom constraint function in `sklearn`.  To optimise the cost function let's use `scipy.optimize.fmin_tnc`. To calculate gradient you can use `fprime` attribute):
1. Logistic Regression
2. L2 penalisation
3. **Individual** Fairness Constrained

When you are finished with the implementation - you should evaluate performance on multiple choices of fairness weight, $\lambda$.

#### Detailed breakdown
The INDIVIDUAL constraint constraint looks like this:

$$ 
f(\beta,S) = \left( \frac{1}{n_1 n_2} \sum_{(x_i,y_i)\in S_1, (x_j,y_j)\in S_2} d(y_i,y_j) (\beta^T  \textbf{x}_i - \beta^T \textbf{x}_j)^2  \right) 
$$


For the constrained optimization we have to solve a problem on the form:

$$ \min_\beta \left( \ell (\beta,S) + \lambda f(\beta,S)  +\gamma \Vert \beta \Vert_2 \right) $$ 

where $\ell$ is some loss function, $f$ is the constraint function, and the $\gamma \Vert \beta \Vert_2 $ is L2 regularization (we use it to avoid overfitting).
(Basically we are minimizing the Lagrangian $\mathscr{L} = \ell (\beta,S) + \lambda f(\beta,S)  +\gamma \Vert \textbf{x} \Vert_2$ with respect to $\beta$ - in ML literature $\mathscr{L}$ is often denoted as J)

Because we are doing classification we are going to use logistic regression. The log loss function is:
$$
\ell = \frac{1}{m}\sum_i^m\left[ -y_i \log(g(x_i)) - (1-y_i)\log(1-g(x_i)) \right], \text{where } g(x_i) = \frac{1}{1+\exp(-\beta_i x_i)}
$$

For the distance function we follow the approach from Berk et al. (2017) and set:
$$d(y_i,y_j) = \begin{cases}
            1, &         \text{if } y_i=y_j,\\
            0, &         \text{if } y_i\neq y_j.
    \end{cases}$$
    
To minimize the total loss function we also need to estimate the gradient of $\mathscr{L}$ with respect to $\beta$. Here to update the $\beta$ values we are just going the gradient's without the fairness constraing - this will make our lives considerably easier. The j'th element of the gradiend is defined as follows:
$$
\frac{\partial \mathscr{L}}{\partial \beta_j} \approx \frac{1}{m}\left( \sum_i  (g(x_i) - y_i) x[j] \right)+ 2\gamma \beta_j
$$

##### A little clarification and tips:
1. In order to simplify the exercise - we cut some corners. *Ideally* we should calculate the gradient in respect to the *individual fairness*. The gradient takes into the account only logistic and l2 loss (aka, parameters are updated based on those). At the same time, our *cost* has a *individual fairness* included. When the update of the parameters stops decreasing the cost, the `fmin_tnc` is going to stop optimisation. So our implementation is not entirely correct.
2. In case you want to have a more correct implementation, you can do `opt.fmin_tnc(func=compute_cost, x0=betas, fprime = None, approx_grad= True, ...)`. It is quite long, but you can still do it
3. I also suggest setting `ftol=1e-5`. 
4. Don not apply l2-regularization on the intercept (when you calculate the gradient).
5. You should include $x_0 = 1$ in your data, for each observation (when it comes to the manual implementation of logistic regression) to include bias (i.e. weight $\beta_0$).
6. To keep the exercise simpler, let's fix $\gamma = 1e-5$.
7. Try $lambda$ is a range from around $1$ to $1e5$.

In [8]:
def sigmoid(x):
    """
    This is logistic regression
    f = 1/(1+exp(-beta^T * x))
    This function assumes as input that you have already multiplied beta and X together
    """
    NotImplemented

def logistic_loss(y_true, y_pred, eps = 1e-10):
    """
    Loss for the logistic regression, y_preds are probabilities
    eps: epsilon for stability
    """
    NotImplemented

def l2_loss(beta):
    """
    L2-Regularisation
    """
    NotImplemented

def fair_loss(y, y_pred, groups):
    """
    Group fairness Loss
    """
    n = y.shape[0]
    n1 = np.sum(groups == 1)
    n2 = np.sum(groups == 2)
    cost = 0
    NotImplemented
    return (cost/(n1*n2))

def compute_gradient(beta,X,y, groups, _lambda,_gamma):
    """Calculate the gradient - used for finding the best beta values. 
       You do not need to use groups and lambda (fmin_tnc expects same input as in func, that's why they are included here)"""
    grad = np.zeros(beta.shape)
    NotImplemented

    for i in range(len(grad)):
        if i == 0: # we do not want to regularize the intercept
            grad[i] =  ...
        else:
            grad[i] = ...
        
    return grad

def compute_cost(beta ,X,y, groups, _lambda, _gamma):
    """Computes cost function with constraints"""
    NotImplemented
    probs = sigmoid(X.dot(beta))
    loss = logistic_loss(y, probs) + _lambda * fair_loss(y,X.dot(beta), groups) + _gamma * l2_loss(beta[1:])
    return loss

In [ ]:
# Run cost with random beta-values and parameters
compute_cost(
    beta = np.random.rand(X_train.shape[1]),
    X = X_train, 
    y = y_train,
    groups = group_train, 
    _gamma = 1, 
    _lambda = 1000
)

In [ ]:
# Run optimization with single `lambda` and `gamma` values
### Set seed and define params
np.random.seed(0)
beta = np.random.rand(X_train.shape[1])
lambda_ = # `1000` worked for robustly scaled continuous features
gamma_ = # `1e-5` worked for robustly scaled continuous features

### Run optimization
result, _, _ = opt.fmin_tnc(
    func=compute_cost,
    x0=beta,
    fprime=compute_gradient,
    maxfun = 500,
    args = (
        X_train, 
        y_train,
        group_train,
        lambda_, 
        gamma_
    ),
    xtol=1e-7,
    ftol=1e-5
)

In [ ]:
import numpy as np
import scipy.optimize as opt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Define fairness constraint function
def fairness_constraint(weights, X, y, sensitive_feature):
    probs = 1 / (1 + np.exp(-X @ weights))  # Sigmoid function
    group_0 = sensitive_feature == 0
    group_1 = sensitive_feature == 1
    fairness_gap = probs[group_0].mean() - probs[group_1].mean()
    return fairness_gap

# Gradient of fairness constraint
def fairness_gradient(weights, X, y, sensitive_feature):
    probs = 1 / (1 + np.exp(-X @ weights))
    group_0 = sensitive_feature == 0
    group_1 = sensitive_feature == 1
    grad = np.mean(X[group_0] * probs[group_0][:, None], axis=0) - np.mean(X[group_1] * probs[group_1][:, None], axis=0)
    return grad

# Logistic loss function
def logistic_loss(weights, X, y):
    probs = 1 / (1 + np.exp(-X @ weights))
    return -np.mean(y * np.log(probs) + (1 - y) * np.log(1 - probs))

# Gradient of logistic loss
def logistic_gradient(weights, X, y):
    probs = 1 / (1 + np.exp(-X @ weights))
    return np.dot(X.T, (probs - y)) / len(y)

# Load and preprocess the data (assuming X, y, and sensitive feature are ready)
X_train, X_test, y_train, y_test, s_train, s_test = train_test_split(X, y, sensitive_feature, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Optimize using constrained minimization
initial_weights = np.zeros(X_train.shape[1])
solution = opt.fmin_tnc(
    func=logistic_loss, 
    x0=initial_weights, 
    fprime=logistic_gradient,
    args=(X_train, y_train),
    approx_grad=False,
    constraints={'type': 'eq', 'fun': fairness_constraint, 'jac': fairness_gradient, 'args': (X_train, y_train, s_train)}
)

# Extract optimized weights
optimal_weights = solution[0]

# Evaluate fairness-optimized model
pred_probs = 1 / (1 + np.exp(-X_test @ optimal_weights))
pred_labels = (pred_probs > 0.5).astype(int)

# Calculate fairness metrics
group_0 = s_test == 0
group_1 = s_test == 1
fairness_difference = pred_probs[group_0].mean() - pred_probs[group_1].mean()
print("Fairness Difference:", fairness_difference)


In [ ]:
import numpy as np

def sigmoid(x):
    """
    Logistic regression function:
    f = 1 / (1 + exp(-x))
    Assumes input is already multiplied by beta.
    """
    return 1 / (1 + np.exp(-x))

def logistic_loss(y_true, y_pred, eps=1e-10):
    """
    Computes the logistic loss.
    Args:
        y_true: Ground truth labels (0 or 1).
        y_pred: Predicted probabilities.
        eps: Small value for numerical stability.
    """
    y_pred = np.clip(y_pred, eps, 1 - eps)  # Prevent log(0) errors
    return -np.mean(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))

def l2_loss(beta):
    """
    L2 Regularization loss.
    Args:
        beta: Model coefficients (excluding intercept).
    """
    return np.sum(beta ** 2)

def fair_loss(y, y_pred, groups):
    """
    Computes group fairness loss based on demographic parity.
    Ensures similar predictions across groups.
    Args:
        y: True labels.
        y_pred: Predicted probabilities.
        groups: Group membership (e.g., 1 for males, 2 for females).
    """
    n1 = np.sum(groups == 1)
    n2 = np.sum(groups == 2)

    if n1 == 0 or n2 == 0:
        return 0  # Avoid division by zero if a group is missing

    group1_mean = np.mean(y_pred[groups == 1])
    group2_mean = np.mean(y_pred[groups == 2])

    return (group1_mean - group2_mean) ** 2 / (n1 * n2)

def compute_gradient(beta, X, y, groups, _lambda, _gamma):
    """
    Computes gradient for optimizing logistic regression with fairness constraint.
    Args:
        beta: Model coefficients.
        X: Feature matrix.
        y: Target labels.
        groups: Group membership.
        _lambda: Fairness regularization parameter.
        _gamma: L2 regularization parameter.
    """
    n = X.shape[0]
    y_pred = sigmoid(X.dot(beta))

    grad = np.zeros_like(beta)
    error = y_pred - y

    grad = X.T.dot(error) / n  # Gradient of logistic loss

    # Fairness gradient
    fair_grad = np.zeros_like(beta)
    n1, n2 = np.sum(groups == 1), np.sum(groups == 2)

    if n1 > 0 and n2 > 0:
        group1_idx = (groups == 1)
        group2_idx = (groups == 2)

        fair_grad = (np.mean(X[group1_idx], axis=0) - np.mean(X[group2_idx], axis=0)) / (n1 * n2)

    # L2 regularization (excluding intercept)
    grad[1:] += _gamma * beta[1:]

    # Add fairness constraint gradient
    grad += _lambda * fair_grad

    return grad

def compute_cost(beta, X, y, groups, _lambda, _gamma):
    """
    Computes cost function with fairness and regularization constraints.
    Args:
        beta: Model coefficients.
        X: Feature matrix.
        y: Target labels.
        groups: Group membership.
        _lambda: Fairness regularization parameter.
        _gamma: L2 regularization parameter.
    """
    probs = sigmoid(X.dot(beta))
    loss = logistic_loss(y, probs)
    
    # Add fairness loss and L2 regularization
    loss += _lambda * fair_loss(y, probs, groups) + _gamma * l2_loss(beta[1:])
    
    return loss
